In [ ]:
import warnings
warnings.filterwarnings('ignore')
from utils import patch_generator, supres_generator, Segmentation, class_generator, vectors_generator, write_to_shape
from tqdm import tqdm
import matplotlib.pyplot as plt

tif_10m='demo/sent2020_10m_.tif'
tif_20m='demo/sent2020_20m_.tif'
s1tif_20m='demo/landusing18_20_1b_.tif'
vectors="zip://demo/test_poly_.zip"

init_params={'tif_10m':tif_10m,'tif_20m':tif_20m,'vectors':vectors,'s1tif_20m':s1tif_20m}

In [5]:
# creates list of datasets (splitting singles vectors and rasters patches into appropriate units)
nit=0
next_item=patch_generator(init_params)
items=[]
for i in tqdm(next_item):
    nit+=1
    items.append(i)
# items.sort(key=(lambda it: it['patch_10'].shape[1]*it['patch_10'].shape[2]))

6it [00:01,  4.36it/s]


In [6]:
# makes superresolution of rgb images
next_super=supres_generator(items)
super_items=[]
for i in tqdm(range(nit)):
    super_items.append(next(next_super))

100%|██████████| 6/6 [00:08<00:00,  1.35s/it]


In [8]:
# makes contour segmentation of superresoluted images
segm_class=Segmentation(line_thresh=0.002,area_thresh=96)
next_segment=segm_class.segm_map_generator(super_items)
segment_items=[]
for i in tqdm(range(nit)):
    segment_items.append(next(next_segment))

100%|██████████| 6/6 [00:14<00:00,  2.47s/it]


In [9]:
# classifies segments into vegetation classes
next_classified=class_generator(segment_items)
classified_items=[]
for i in tqdm(range(nit)):
    classified_items.append(next(next_classified))

100%|██████████| 6/6 [00:01<00:00,  4.81it/s]


In [ ]:
# vectorises segments
next_vectorized=vectors_generator(classified_items)
vectorized_items=[]
for i in tqdm(range(nit)):
    vectorized_items.append(next(next_vectorized))

In [ ]:
# writes vectors to a file
path_to_output='trash1.shp'
write_to_shape(vectorized_items, path_to_output)

In [10]:
# a glance at the results
from ipywidgets import interact
from skimage.segmentation import mark_boundaries

nm= 10 if len(segment_items)>10 else len(segment_items)-1
param_dict=segment_items[nm]

@interact
def get_by_num(number=(0,nm,1)):    
    param_dict=segment_items[int(number)]     
    rgb10=param_dict['rgb_10']    
    rgb5=param_dict['rgb_5']
    segm=mark_boundaries(rgb5, param_dict['segments_5'])   
    cl=param_dict['classes']

    fig, axs = plt.subplots(1, 4, figsize=(25, 20))    
    for num, im in enumerate(zip(['origin 10 m', 'superresuted', 'segmented', 'classified'],[rgb10, rgb5, segm, cl])):
        axs[num].set(title=im[0], xticks=[], yticks=[])
        axs[num].imshow(im[1])
    plt.show(fig)        
        

interactive(children=(IntSlider(value=2, description='number', max=5), Output()), _dom_classes=('widget-intera…